In [ ]:
from azureml.core import Workspace, Experiment

vrk_bank_ws = Workspace.from_config()
vrk_bank_exp = Experiment(workspace=ws, name="vrk_bank_train_exp")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

# Choose a name for your CPU cluster
bnkcompute_cluster_name = "vrk-bnk-cpu-clst4"

# Verify that cluster does not exist already
try:
    vrk_bnk_aml_compute = ComputeTarget(workspace=vrk_bank_ws, name=bnkcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    vrk_bnk_compute = ComputeTarget.create(ws, bnkcompute_cluster_name, compute_config)

vrk_bnk_compute.wait_for_completion(show_output=True)

In [ ]:
#Create an environment: Define a conda environment YAML file that script is depeneded on and use it in environment.
%%writefile conda_dependencies.yml

dependencies:
    -python=3.6.2
    -scikit-learn
    -pip
     - azureml-defaults

In [ ]:
from azureml.core import Environment

sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='./conda_dependencies.yml')

In [ ]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
from azureml.core import ScriptRunConfig
import os

# Specify parameter sampler
param_sampling_logistic_regre = RandomParameterSampling( {
        "--C": choice(0.001, 0.01, 0.1, 1, 10, 100),
        "--max_iter": choice(50, 100, 200, 400, 600, 900, 1000)
    }
)

# Specify a Policy
early_termination_policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=1, delay_evaluation=5)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
bnk_src = ScriptRunConfig(source_directory='.',
                      script='train.py',
                      arguments=['--C', '--max_iter'],
                      compute_target=vrk_bnk_compute,
                      environment=sklearn_env)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
bnk_hyperdrive_config = HyperDriveConfig(run_config=bnk_src,
                                     hyperparameter_sampling=param_sampling_logistic_regre,
                                     policy=early_termination_policy ,
                                     primary_metric_name="Accuracy",
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=12,
                                     max_concurrent_runs=4)

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

bnk_hyperdrive_run = vrk_bank_exp.submit(bnk_hyperdrive_config)

In [ ]:
# Monitor hyper drive runs
from azureml.widgets import RunDetails
RunDetails(bnk_hyperdrive_run).show()

In [ ]:
# wait for completion
bnk_hyperdrive_run.wait_for_completion(show_output=True)

In [ ]:
assert(bnk_hyperdrive_run.get_status() == "Completed")

In [ ]:
best_run = bnk_hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['Arguments']
print("\n Best parameter values:", parameter_values)
print('\n Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])
print('\n C:',parameter_values[0])
print('\n max_iter:',parameter_values[1])

In [ ]:
print(best_run.get_file_names())

In [ ]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
model = joblib.load('outputs/model.joblib')

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

bankmarketingauto_webpath = [
                          'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'
                        ]

#create bankmarketing data set in tabular format using TabularDatasetFactory
bankmarketingauto_dataset = TabularDatasetFactory.from_delimited_files(path=bankmarketingauto_webpath)


In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(bankmarketingauto_dataset)
x = x.join(y)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
vrk_bnk_automl_config = AutoMLConfig(
                   experiment_timeout_minutes=30,
                   task="classification",
                   primary_metric="Accuracy",
                   training_data=x,
                   label_column_name='y',
                   n_cross_validations=5)

In [2]:
# Submit your automl run
from azureml.pipeline.steps import AutoMLStep
automl_bnk_run = vrk_bank_exp.submit(vrk_bnk_automl_config, show_output=True)

### YOUR CODE HERE ###

In [ ]:
automl_bnk_run.wait_for_completion()

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_run_model, best_fitted_model = automl_bnk_run.get_output()
print(best_fitted_model.get_file_names())